# Tutorial 1: Hello Spark - Your First Node

## Welcome to Spark ADK! 🚀

Welcome to your first Spark tutorial! In this hands-on session, you'll learn the foundation of Spark ADK by creating and running your first nodes.

### What You'll Learn

By the end of this tutorial, you'll be able to:
- ✅ Understand what a **Node** is in Spark
- ✅ Create and execute a simple node
- ✅ Understand the `process()` method and its role
- ✅ Work with inputs, outputs, and ExecutionContext
- ✅ Return data from nodes

### Prerequisites
- Basic Python knowledge
- Python 3.12+ installed
- Spark ADK installed (`pip install spark`)

### Time Required
⏱️ Approximately 15-20 minutes

---

## 🧠 What is a Node?

In Spark ADK, a **Node** is the fundamental building block of your AI systems. Think of a node as a processing unit that:

1. **Receives inputs** (optional)
2. **Processes data** (performs some work)
3. **Produces outputs** (optional)

### Node Class Hierarchy

```
BaseNode (abstract base class)
    ↓
Node (adds configuration, capabilities, queues)
    ↓
Your Custom Nodes
```

### Key Concepts

- **BaseNode**: The abstract foundation all nodes inherit from
- **Node**: The concrete class you typically inherit from (adds features like config, state, queues)
- **process() method**: Where you define what your node does
- **ExecutionContext**: Container for inputs, outputs, state, and metadata


## Example 1: The Simplest Node

Let's start with the absolute simplest node possible - one that just prints a message.

### Key Points:
- Inherit from `Node` class
- Define a `process()` method
- That's it! No other methods required.

In [ ]:
from spark.nodes import Node
from spark.utils.common import arun

In [ ]:
class SimpleNode(Node):
    """The simplest possible node - just prints a greeting."""
    
    def process(self):
        print("Hello from Spark Node!")
        print("This is your first node 🎉")

### Running a Node

There are multiple ways to execute a node:

1. **node.do()** - Direct execution (returns NodeMessage)
2. **node.go()** - Continously run node (take inputs from a queue, does not return anything.)
3. **node.run()** - Wrapper for do() or go() 

All three are async methods and need to be 'await'ed.

In Jupyter notebooks (which support async), we can use `await` directly.   In Actual application, we need to use asyncio.run.  Spark provide a convenient function `arun` that does `asyncio.run`.

In [ ]:
# Create an instance of our node
node = SimpleNode()

# Execute it
result = await node.do()

print(f"\nReturn value: {result}")
print(f"Return type: {type(result)}")

### Understanding the Return Value

Notice that even though our `process()` method doesn't explicitly return anything, the node execution returns a `NodeMessage` object. This is Spark's standardized message format.

---

## Example 2: Node with Return Value

Most nodes will return data for other nodes to use. Let's create a node that performs a calculation and returns the result.

### Key Point:
- The `process()` method can return any Python value (str, int, dict, list, etc.)
- Returned values are automatically wrapped in NodeMessage format

In [ ]:
class CalculatorNode(Node):
    """A node that performs a calculation and returns the result."""
    
    def process(self):
        result = 42 * 2
        print(f"Calculating: 42 * 2 = {result}")
        return result  # Return a simple integer

In [ ]:
calc_node = CalculatorNode()
result = await calc_node.do()

print(f"\nResult: {result}")
print(f"Result content: {result.content}")

---

## Example 3: Node with Dictionary Return

When you need to return multiple pieces of information, use a dictionary. This is the most common pattern in Spark.

### Why Dictionaries?
- Named keys make it clear what each value represents
- Easy to pass multiple values to subsequent nodes
- Standard pattern in Spark for structured data

In [ ]:
class DataProcessorNode(Node):
    """A node that returns structured data as a dictionary."""
    
    def process(self):
        # Process some data
        data = {
            'status': 'success',
            'value': 100,
            'message': 'Data processed successfully',
            'items': ['item1', 'item2', 'item3']
        }
        return data

In [ ]:
data_node = DataProcessorNode()
result = await data_node.do()

print(f"Result: {result.content}")
print(f"\nStatus: {result.content['status']}")
print(f"Value: {result.content['value']}")
print(f"Items: {result.content['items']}")

---

## Example 4: Node with Inputs (ExecutionContext)

Real nodes need to receive data from somewhere! The `ExecutionContext` is how nodes receive inputs.

### ExecutionContext Components:
- **`context.inputs`** - Data passed to the node
- **`context.state`** - Persistent state across executions
- **`context.metadata`** - Additional information about execution
- **`context.outputs`** - Accumulated outputs (read-only in process)

### Important Note:
While Spark tolerates `process()` methods without arguments for convenience, **always write** `process(self, context)` to access inputs properly.

In [ ]:
class GreetingNode(Node):
    """A node that uses input data to create a personalized greeting."""
    
    def process(self, context):
        # Access inputs from the context
        name = context.inputs.content.get('name', 'Stranger')
        language = context.inputs.content.get('language', 'English')
        
        # Create greeting based on language
        greetings = {
            'English': f'Hello, {name}!',
            'Spanish': f'¡Hola, {name}!',
            'French': f'Bonjour, {name}!',
            'German': f'Guten Tag, {name}!',
        }
        
        greeting = greetings.get(language, f'Hello, {name}!')
        print(greeting)
        
        return {
            'greeting': greeting,
            'name': name,
            'language': language
        }

In [ ]:
# Create node and pass inputs directly to do()
greeting_node = GreetingNode()

# Execute with inputs
result1 = await greeting_node.do({'name': 'Alice', 'language': 'Spanish'})
print(f"Result: {result1.content}\n")

result2 = await greeting_node.do({'name': 'Bob', 'language': 'French'})
print(f"Result: {result2.content}\n")

result3 = await greeting_node.do({'name': 'Charlie'})  # Default language
print(f"Result: {result3.content}")

---

## Example 5: Node with State

Nodes can maintain state across multiple executions. State is persistent within the node instance.

### When to Use State:
- Counting operations
- Accumulating results
- Tracking history
- Maintaining context across calls

In [ ]:
class CounterNode(Node):
    """A node that counts how many times it has been executed."""
    
    def process(self, context):
        # Initialize counter in state if not exists
        if 'count' not in context.state:
            context.state['count'] = 0
        
        # Increment counter
        context.state['count'] += 1
        
        current_count = context.state['count']
        print(f"Execution #{current_count}")
        
        return {
            'execution_number': current_count,
            'message': f'This node has been called {current_count} time(s)'
        }

In [ ]:
counter_node = CounterNode()

# Call it multiple times
for i in range(5):
    result = await counter_node.do()
    print(f"  → {result.content['message']}\n")

---

## Example 6: Practical Node - Text Processor

Let's create a more practical example: a text processing node that performs various transformations.

This demonstrates:
- Multiple input parameters
- Conditional logic
- Different operations based on inputs
- Comprehensive return values

In [ ]:
class TextProcessorNode(Node):
    """A node that performs various text transformations."""
    
    def process(self, context):
        text = context.inputs.content.get('text', '')
        operation = context.inputs.content.get('operation', 'none')
        
        if not text:
            return {'error': 'No text provided'}
        
        # Perform the requested operation
        operations = {
            'uppercase': text.upper(),
            'lowercase': text.lower(),
            'reverse': text[::-1],
            'word_count': len(text.split()),
            'char_count': len(text),
            'title': text.title(),
            'none': text
        }
        
        result = operations.get(operation, text)
        
        return {
            'original': text,
            'operation': operation,
            'result': result,
            'original_length': len(text)
        }

In [ ]:
processor = TextProcessorNode()

# Test different operations
test_text = "Hello World from Spark ADK"

operations = ['uppercase', 'lowercase', 'reverse', 'word_count', 'title']

for op in operations:
    result = await processor.do({'text': test_text, 'operation': op})
    print(f"{op.upper():12} → {result.content['result']}")

---

## 🔍 Async vs Sync in Spark

### Important Notes:

1. **You can write sync or async `process()` methods**
   ```python
   def process(self, context):        # Sync - OK!
       return {'result': 42}
   
   async def process(self, context):  # Async - Preferred!
       return {'result': 42}
   ```

2. **Spark auto-wraps sync methods to async**
   - The framework handles this automatically
   - But prefer writing async for consistency

3. **Execution methods are always async**
   - `await node.do()`
   - `await node.run()`
   - Use `arun()` utility if calling from sync context

---

## 💪 Practice Exercises

Now it's your turn! Try these exercises to solidify your understanding.

### Exercise 1: Temperature Converter

Create a node that converts temperatures between Celsius and Fahrenheit.

**Requirements:**
- Accept `temperature` and `from_unit` ('C' or 'F') as inputs
- Convert to the other unit
- Return both values and the conversion formula used

**Formulas:**
- C to F: `(C × 9/5) + 32`
- F to C: `(F - 32) × 5/9`

In [ ]:
class TemperatureConverterNode(Node):
    """Your code here!"""
    
    def process(self, context):
        # TODO: Implement temperature conversion
        pass

# Test your node
# temp_node = TemperatureConverterNode()
# result = await temp_node.do({'temperature': 100, 'from_unit': 'C'})
# print(result.content)

### Exercise 2: Statistics Node

Create a node that calculates basic statistics for a list of numbers.

**Requirements:**
- Accept a list of numbers as input
- Calculate: mean, median, min, max, sum
- Return all statistics in a dictionary

In [ ]:
class StatisticsNode(Node):
    """Your code here!"""
    
    def process(self, context):
        # TODO: Implement statistics calculation
        pass

# Test your node
# stats_node = StatisticsNode()
# result = await stats_node.do({'numbers': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})
# print(result.content)

### Exercise 3: Accumulator Node

Create a node that accumulates values over multiple calls using state.

**Requirements:**
- Accept a number as input
- Add it to a running total (stored in state)
- Return the current total and count of additions
- Include a 'reset' input option to clear the state

In [ ]:
class AccumulatorNode(Node):
    """Your code here!"""
    
    def process(self, context):
        # TODO: Implement accumulator with state
        pass

# Test your node
# acc_node = AccumulatorNode()
# await acc_node.do({'value': 10})
# await acc_node.do({'value': 20})
# result = await acc_node.do({'value': 30})
# print(result.content)  # Should show total: 60, count: 3

---

## ✅ Solutions

Try the exercises yourself first! Solutions are provided below.

In [ ]:
# Solution 1: Temperature Converter
class TemperatureConverterNode(Node):
    def process(self, context):
        temp = context.inputs.content.get('temperature')
        from_unit = context.inputs.content.get('from_unit', 'C').upper()
        
        if temp is None:
            return {'error': 'Temperature value required'}
        
        if from_unit == 'C':
            converted = (temp * 9/5) + 32
            to_unit = 'F'
            formula = '(C × 9/5) + 32'
        elif from_unit == 'F':
            converted = (temp - 32) * 5/9
            to_unit = 'C'
            formula = '(F - 32) × 5/9'
        else:
            return {'error': 'Invalid unit. Use C or F'}
        
        return {
            'original': f"{temp}°{from_unit}",
            'converted': f"{converted:.2f}°{to_unit}",
            'formula': formula
        }

# Test
temp_node = TemperatureConverterNode()
result = await temp_node.do({'temperature': 100, 'from_unit': 'C'})
print(f"Solution 1: {result.content}")

In [ ]:
# Solution 2: Statistics Node
class StatisticsNode(Node):
    def process(self, context):
        numbers = context.inputs.content.get('numbers', [])
        
        if not numbers:
            return {'error': 'No numbers provided'}
        
        sorted_nums = sorted(numbers)
        n = len(sorted_nums)
        
        # Calculate median
        if n % 2 == 0:
            median = (sorted_nums[n//2 - 1] + sorted_nums[n//2]) / 2
        else:
            median = sorted_nums[n//2]
        
        return {
            'count': n,
            'sum': sum(numbers),
            'mean': sum(numbers) / n,
            'median': median,
            'min': min(numbers),
            'max': max(numbers),
            'range': max(numbers) - min(numbers)
        }

# Test
stats_node = StatisticsNode()
result = await stats_node.do({'numbers': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]})
print(f"Solution 2: {result.content}")

In [ ]:
# Solution 3: Accumulator Node
class AccumulatorNode(Node):
    def process(self, context):
        # Check for reset command
        if context.inputs.content.get('reset'):
            context.state.clear()
            return {'message': 'Accumulator reset', 'total': 0, 'count': 0}
        
        # Initialize state
        if 'total' not in context.state:
            context.state['total'] = 0
            context.state['count'] = 0
        
        # Add new value
        value = context.inputs.content.get('value', 0)
        context.state['total'] += value
        context.state['count'] += 1
        
        return {
            'total': context.state['total'],
            'count': context.state['count'],
            'average': context.state['total'] / context.state['count'],
            'last_added': value
        }

# Test
acc_node = AccumulatorNode()
await acc_node.do({'value': 10})
await acc_node.do({'value': 20})
result = await acc_node.do({'value': 30})
print(f"Solution 3: {result.content}")

---

## 🎯 Key Takeaways

Congratulations! You've learned the fundamentals of Spark nodes. Let's recap:

### ✅ What You Learned:

1. **Nodes are the building blocks** of Spark systems
   - Inherit from `Node` class
   - Define a `process()` method

2. **The process() method** is where the magic happens
   - Can be sync or async
   - Optionally accepts ExecutionContext
   - Can return any Python value

3. **ExecutionContext provides**:
   - `context.inputs` - Data passed to the node
   - `context.state` - Persistent state across executions
   - `context.metadata` - Execution metadata

4. **Execution methods**:
   - `await node.do(inputs)` - Direct execution
   - `await node.run(inputs)` - Higher-level execution
   - Pass inputs as dictionaries

5. **Best practices**:
   - Use dictionaries for structured returns
   - Always write `process(self, context)` for clarity
   - Prefer async methods
   - Use state for persistent data

### 📚 Related Documentation:
- Example file: `examples/e001_hello.py`
- Source: `spark/nodes/base.py`, `spark/nodes/nodes.py`

---

## 🚀 Next Steps

Now that you understand individual nodes, you're ready to connect them together!

### Coming Up in Tutorial 2:
- **Connecting nodes** into workflows
- **Creating graphs** with the `>>` operator
- **Passing data** between nodes
- **Building pipelines** that solve real problems

### Challenge Before Next Tutorial:
Try creating a node that:
1. Accepts a URL as input
2. Fetches content from that URL (use `requests` library)
3. Returns word count and character count
4. Uses state to track how many URLs have been processed

This will prepare you well for the next tutorial!

---

### 🎓 Tutorial Series:
- ✅ **Tutorial 1: Hello Spark** (You are here)
- ➡️ Tutorial 2: Simple Flows and Graph Basics
- Tutorial 3: Your First AI Agent
- Tutorial 4: Conditional Routing and Decision Making
- ...and more!

---

**Questions or Issues?** Check the Spark documentation or open an issue on GitHub.

Happy building with Spark! 🚀